In [ ]:
import torch
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from audioop import mul
import numpy as np
import torch
from sklearn import preprocessing
import os
from pathlib import Path
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from sklearn.preprocessing import OneHotEncoder

In [ ]:
torch.cuda.FloatTensor(10,1).normal_(0,1)

In [ ]:
# image_feature
vis_emb = np.load(r"C:\HolisticVideoUnderstanding\sampled_test\applying cream\0PgfOKP1Ow0_66\InceptionResnetV2_MaxPooling.npz")["InceptionResnetV2"]
print(vis_emb.shape)

# textual_feature
txt_emb = np.load(r"C:\HolisticVideoUnderstanding\sampled_test\applying cream\0PgfOKP1Ow0_66\Elmo_Mean.npz")["Elmo"]
print(txt_emb.shape)

In [ ]:
# Dataset Loader
class HVUDataset(Dataset):
    def __init__(self, n_trial):
        self.total = n_trial
        
    def __len__(self):        
        return self.total # 4
    
    # this function will return one n-way, k-shot set
    def __getitem__(self,idx):
        
        #return torch.randperm(idx)
        return "erol___"+str(idx)

In [ ]:
ds = HVUDataset(n_trial=30)
    
train_loader = DataLoader(dataset= ds, batch_size = 10, shuffle=True, num_workers = 0) 

In [ ]:
for local_batch in train_loader:
    print(local_batch)    

In [ ]:
empty = []
empty_test = []

a = [np.ones(3)*1, np.ones(3)*2, np.ones(3)*3, np.ones(3)*4]
b = [np.ones(3)*10, np.ones(3)*20, np.ones(3)*30, np.ones(3)*40]

empty.extend(a[:-1])
empty_test.extend(a[-1:])

empty.extend(b[:-1])
empty_test.extend(b[-1:])


print(empty)
print(empty_test)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

normalizer = MinMaxScaler()

In [ ]:
visual_f = np.load("C:/HolisticVideoUnderstanding/uniform_train/air drumming/qF1dEUpUbNA_80/InceptionResnetV2_MaxPooling.npz")['InceptionResnetV2']
textual_f = np.load("C:/HolisticVideoUnderstanding/uniform_train/air drumming/qF1dEUpUbNA_80/Elmo_Mean.npz")['Elmo']

In [ ]:
np.mean(visual_f), np.min(visual_f), np.max(visual_f)

In [ ]:
new_visual_f = normalizer.fit_transform(np.float32(visual_f).reshape(-1, 1))

In [ ]:
np.mean(new_visual_f), np.min(new_visual_f), np.max(new_visual_f)

In [ ]:
np.mean(textual_f), np.min(textual_f), np.max(textual_f)

In [ ]:
new_textual_f = normalizer.fit_transform(np.float32(textual_f).reshape(-1, 1))

In [ ]:
np.mean(new_textual_f), np.min(new_textual_f), np.max(new_textual_f)

#### 1) Distribution Aware Classification

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
gt = [0,0,0,0,0, 1,1,1,1,1,2,2,2,2,2]
n_way = 3
k_shot = 5

# Training Stage
mu_1 = torch.normal(10, 1, (5,64))
mu_2 = torch.normal(-30, 1, (5,64))
mu_3 = torch.normal(170, 1, (5,64))

logvar_1 = torch.normal(10, 1, (5,64))
logvar_2 = torch.normal(-30, 1, (5,64))
logvar_3 = torch.normal(170, 1, (5,64))

mus = torch.cat((mu_1, mu_2, mu_3))
logvars = torch.cat((logvar_1, logvar_2, logvar_3))

# Testing Stage
mu_1 = torch.normal(7, 1, (5,64))
mu_2 = torch.normal(-20, 1, (5,64))
mu_3 = torch.normal(100, 1, (5,64))

logvar_1 = torch.normal(7, 1, (5,64))
logvar_2 = torch.normal(-20, 1, (5,64))
logvar_3 = torch.normal(100, 1, (5,64))

test_mus = torch.cat((mu_1, mu_2, mu_3))
test_logvars = torch.cat((logvar_1, logvar_2, logvar_3))

#mus = torch.rand((15,1,64)).reshape((n_way*k_shot, 64))
#logvars = torch.rand((15,1,64)).reshape((n_way*k_shot, 64))
#test_mus = torch.rand((15,1,64)).reshape((n_way*k_shot, 64))
#test_logvars = torch.rand((15,1,64)).reshape((n_way*k_shot, 64))

In [ ]:
mus.shape, logvars.shape, test_mus.shape, test_logvars.shape

##### 1.a) Class mus, logvar construction stage

In [ ]:
alpha = 1 / k_shot
avg_train_mus = []
avg_train_vars = []

In [ ]:
for i in range(0,len(mus) - 1, k_shot):
    
    avg_mu = np.mean(mus[i: i + k_shot].cpu().detach().numpy(), axis=0)
    
    class_mu = mus[i: i + k_shot].cpu().detach().numpy()
    class_variance = logvars[i: i + k_shot].cpu().detach().numpy()
        
    # sinif ici her bir mu variance al
    avg_var = [0.0] * 64
    for j in range(len(class_variance)):
        
        # For the 1st, 2nd, 3rd solutions            
        if k_shot != 1:
            avg_var += (( 1/k_shot * class_variance[j] ) + (1/k_shot * ((class_mu - avg_mu)**2)))
        # If there is only 1 sample, avg_var equals to that sample's var
        else:
            avg_var += class_variance[j]        
            
    # class oriented var
    avg_train_vars.append(avg_var)
    
    # class oriented mu
    avg_train_mus.append(avg_mu)

In [ ]:
avg_train_mus[0].shape, avg_train_mus[1].shape, avg_train_mus[2].shape

In [ ]:
avg_train_vars[0].shape, avg_train_vars[1].shape, avg_train_vars[2].shape

##### 2) Testing stage

In [ ]:
len(avg_train_mus)

In [ ]:
loss = torch.nn.CrossEntropyLoss()

gt_tensor = np.array(gt, dtype=np.int32)
gt_tensor = torch.from_numpy(gt_tensor).type(torch.long).reshape(-1,1)


In [ ]:
test_pred_labels = []

for i in range(len(test_mus)):
    test_dist = torch.distributions.Normal(test_mus[i],test_logvars[i])

    local_scores = []
    for j in range(len(avg_train_mus)):
        
        # KL divergence
        train_dist = torch.distributions.Normal(torch.Tensor(avg_train_mus[j]), torch.Tensor(avg_train_vars[j]))
        print("Train Dist")
        print(train_dist)
        scr = torch.distributions.kl_divergence(test_dist, train_dist).mean()
        print("Test Dist")
        print(test_dist)
        #print("{}.th sample, {}.th class, SCORE:{}".format(i,j,scr))
        local_scores.append(scr)
        
    #print(local_scores)
    
    local_similarities = torch.from_numpy(np.array([-1 * elem for elem in local_scores])).reshape(1,-1)
    
    # categorical cross entropy loss    
    #print(local_similarities)    
    #print(gt_tensor[i])
    cc_loss = loss(local_similarities, gt_tensor[i])
    
    # min of kl divergence for class decision
    pred_cls_idx = np.argmin(local_scores)
    
    print(pred_cls_idx)
    print(cc_loss)
    print("----------")
        
    test_pred_labels.append(pred_cls_idx)

In [ ]:
raw = [-0.2025, -1.7995, -3.1739]
raw_exp = [np.exp(elem) for elem in raw]
probabilities = [elem/np.sum(raw_exp) for elem in raw_exp]

probabilities

In [ ]:
print("Pred \t G.T.")
for i in range(len(test_pred_labels)):
    print("{} \t {}".format(test_pred_labels[i], gt[i]))

In [ ]:
print(classification_report(gt, test_pred_labels))

In [14]:
import math
import numpy as np
import matplotlib.pyplot as plt
 
'''
Hypothesis Function - Sigmoid function
'''
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))
 
'''
yHat represents the predicted value / probability value calculated as output of hypothesis / sigmoid function
 
y represents the actual label
'''
def cross_entropy_loss(yHat, y):
    if y == 1:
      return -np.log(yHat)
    else:
      return -np.log(1 - yHat)

In [18]:
gt = [0,1,0]
raw = [-272.06121, -546.5770, -666.3647]
exp = []

for elem in raw:
    exp.append((math.exp(elem)))
    
sum_exp = 0.0

for elem in exp:
    sum_exp += elem
    

probability = []

for elem in exp:
    probability.append(elem / sum_exp)   

sum_loss = 0.0

for gt_elem,prob in zip(gt, probability):
    print(gt_elem)
    print(prob)
    sum_loss += cross_entropy_loss(prob, gt_elem)
    
    
print("-----------")
print(sum_loss)

0
1.0
1
6.015991412996108e-120
0
5.703876292213237e-172
-----------
inf


C:\Users\PC\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in log


In [1]:
from datetime import datetime

In [6]:
now = datetime.now()
current_time = now.strftime("%H_%M_%S")

today = datetime.today()
current_day = today.strftime("%d_%m_%Y_")


print(current_day+current_time)

13_03_2022_16_05_21


In [5]:

print(d1)


13_03_2022


In [3]:
a = 9.7297e-6
b = 1.0832e-6
c = 5.6362e-9

t = a + b + c

In [4]:
p1 = a / t
p2 = b / t
p3 = c / t

p1, p2, p3

(0.8993545725714722, 0.10012445121734677, 0.0005209762111809544)

1.0